In [17]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import joblib
import os

In [8]:
df = pd.read_csv('data/heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [15]:
numeric_cols = ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']
category_cols = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), category_cols)
    ],
    remainder='passthrough' 
)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components=2)),
    ('classifier', GaussianNB(var_smoothing=0.001))
])

X = df.drop('HeartDisease', axis=1)
y = df['HeartDisease']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

accuracy = np.mean(y_pred == y_test)
print(f"Accuracy: {accuracy:.2f}")
print("Sample predictions vs actual:")
print(f"Predicted: {y_pred[:30]}")
print(f"Actual   : {y_test.values[:30]}")

Accuracy: 0.82
Sample predictions vs actual:
Predicted: [0 1 1 1 0 1 1 1 0 1 0 1 1 0 1 1 0 1 0 1 1 1 0 0 1 0 0 1 0 0]
Actual   : [0 1 1 1 0 1 1 0 1 1 0 0 0 0 1 1 0 1 1 0 1 0 0 0 1 1 0 1 0 0]


In [18]:
if not os.path.exists('model'):
    os.makedirs('model')

joblib.dump(pipeline, 'model/heart_disease_model.joblib')

['model/heart_disease_model.joblib']